# AI MedReview Lead Research

In this lesson, you will learn more about Tools. You'll focus on three key elements of Tools:
- Versatility
- Fault Tolerance
- Caching

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM
- [Serper](https://serper.dev)

In [2]:
import os
import openai

In [3]:
# from openai import OpenAI
# client = OpenAI()

# completion = client.chat.completions.create(
#   model="gpt-4o",
#   messages=[
#     {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
#     {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
#   ]
# )

# print(completion.choices[0].message)

In [4]:
import openai

from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [5]:
import os
# from utils import get_openai_api_key, pretty_print_result
# from utils import get_serper_api_key

# openai_api_key = get_openai_api_key()
# os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
# os.environ["SERPER_API_KEY"] = get_serper_api_key()

## Creating Agents

In [6]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify Primary Care Networks (PCN) leads that match the {pcn_list} "
         "our ideal customer profile, I will provide you a list of 5 PCNs in json, to research and obtain contact information for.",
    backstory=(
        "As a part of the dynamic sales team at AI MedReview - the Friends and Family Test analysis tool to help PCNs plan their services, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

In [7]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of AI MedReview's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

## Creating Tools

### crewAI Tools

In [8]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [9]:
directory_read_tool = DirectoryReadTool(directory='./pcn')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

### Custom Tool
- Create a custom tool using crewAi's [BaseTool](https://docs.crewai.com/core-concepts/Tools/#subclassing-basetool) class

In [10]:
from crewai_tools import BaseTool

- Every Tool needs to have a `name` and a `description`.
- For simplicity and classroom purposes, `SentimentAnalysisTool` will return `positive` for every text.
- When running locally, you can customize the code with your logic in the `_run` function.

In [11]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [12]:
sentiment_analysis_tool = SentimentAnalysisTool()

## Creating Tasks

- The Lead Profiling Task is using crewAI Tools.

In [13]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of the 5 PCNs as specified as input "
        "who might have an interest in using tech solutions."
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on the 5 specified PCNs, "
        "including background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
        "Include some of AI MedReview's key features which you can research in the file './pcn/AI MedReview Product Overview.txt, that might be of interest to the PCNs you have researched.'"
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

- The Personalized Outreach Task is using your custom Tool `SentimentAnalysisTool`, as well as crewAI's `SerperDevTool` (search_tool).

In [14]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on PCNs, "
        "craft a personalized outreach campaign "
        "aimed at the clinical director of the pcn, "
        "The campaign should address their recent activity "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with PCNs culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to the each of the PCN's listed in {pcn_list} "
        "specifically targeting the Clinical Director and management lead."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with PCNs corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

## Creating the Crew

In [15]:
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],
	
    verbose=2,
	memory=True
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [16]:
inputs = {
    "pcn_list": ["Watergate PCN", "Truro PCN", "Penwith PCN", "East Cornwall PCN", "St Austell Health PCN",]
}

result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of the 5 PCNs as specified as input who might have an interest in using tech solutions.Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


2024-05-22 07:26:30,795 - 8722615296 - manager.py-manager:281 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-22 07:26:30,795 - 8722615296 - manager.py-manager:281 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-22 07:26:30,796 - 8722615296 - manager.py-manager:281 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')




> Entering new CrewAgentExecutor chain...


2024-05-22 07:26:33,228 - 8722615296 - manager.py-manager:281 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-22 07:26:33,229 - 8722615296 - manager.py-manager:281 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-22 07:26:33,229 - 8722615296 - manager.py-manager:281 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')


To begin the task, I need to list the files in the given directory to locate the PCN information and the AI MedReview Product Overview.

Action: List files in directory
Action Input: {}
 

File paths: 
-./pcn/pcn_list.txt
- ./pcn/AI MedReview Product Overview.txt



...

### Email Draft for Penwith PCN

#### Subject: Enhancing Patient Feedback and Integrated Care at Penwith PCN

#### Email Body:
```
Dear [Clinical Director's Name],

I hope this email finds you well. I am writing to acknowledge the commendable efforts Penwith PCN has made in adopting integrated care models and engaging in local health initiatives to address community needs. Your dedication to improving primary care services through collaboration is truly noteworthy.

At AI MedReview, we are committed to aiding PCNs like Penwith in enhancing patient feedback analysis and supporting integrated care planning. Our Friends and Family Test analysis tool can streamline the collection of patient feedback, providing actionable insights that can further support your integrated care initiatives.

We would love the opportunity to discuss how our solutions can be customized to meet the specific needs of Penwith PCN. I would be happy to schedule a meeting or demo session at your convenience to demons

2024-05-22 07:28:35,982 - 8722615296 - manager.py-manager:281 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-22 07:28:35,983 - 8722615296 - manager.py-manager:281 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-22 07:28:35,983 - 8722615296 - manager.py-manager:281 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')


### Email Draft for East Cornwall PCN

#### Subject: Elevating Service Delivery with AI MedReview at East Cornwall PCN

#### Email Body:
```
Dear [Clinical Director's Name],

I hope this email finds you well. I am writing to commend East Cornwall PCN for its active participation in regional healthcare initiatives and the introduction of new patient care services and support programs. Your focus on delivering high-quality primary care services is truly impressive.

At AI MedReview, we are dedicated to empowering PCNs like East Cornwall with advanced tools for patient feedback collection and analysis. Our Friends and Family Test analysis tool can capture detailed patient insights, providing the data-driven support needed for effective service planning and improvement.

We believe AI MedReview can play a significant role in helping East Cornwall PCN achieve its goals. I would be delighted to schedule a meeting or demo session to showcase how our solutions can be tailored to meet your spec

2024-05-22 07:28:48,925 - 8722615296 - manager.py-manager:281 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-22 07:28:48,926 - 8722615296 - manager.py-manager:281 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-22 07:28:48,926 - 8722615296 - manager.py-manager:281 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')


### Email Draft for St Austell Health PCN

#### Subject: Advancing Integrated Care with AI MedReview at St Austell Health PCN

#### Email Body:
```
Dear [Clinical Director's Name],

I hope this email finds you well. I am reaching out to commend St Austell Health PCN for its recent achievements in developing new health initiatives and patient support programs, as well as your collaboration with local healthcare providers to enhance service delivery. Your dedication to providing comprehensive primary care services is truly exemplary.

At AI MedReview, we are committed to supporting PCNs like St Austell Health in collecting and analyzing patient feedback to drive service improvement. Our Friends and Family Test analysis tool can deliver detailed patient insights, offering the data-driven support necessary for planning and executing integrated care services.

We believe AI MedReview can significantly contribute to St Austell Health PCN's objectives. I would be delighted to schedule a meeti

- Display the final result as Markdown.

In [17]:
from IPython.display import Markdown
Markdown(result)

Personalized Email Drafts for Each PCN

---

### Email Draft for Watergate PCN

#### Subject: Enhancing Patient Care Coordination at Watergate PCN

#### Email Body:
```
Dear Emma Seward-Adams,

I hope this email finds you well. I am writing to express my admiration for the remarkable advancements Watergate PCN has achieved in integrating self-referral services and online resources for bereavement support and brain injury management. Your commitment to delivering proactive, personalized, and coordinated care for approximately 45,000 patients is truly commendable.

At AI MedReview, we are dedicated to supporting PCNs like yours in enhancing patient care and service delivery. Our Friends and Family Test analysis tool can provide valuable insights through detailed patient feedback, helping you to further tailor services and improve outcomes. Additionally, our AI-driven solutions can enhance the coordination of care across various services and practices within your network.

We would love the opportunity to showcase how AI MedReview can help Watergate PCN achieve its goals. I would be delighted to schedule a demo session at your convenience to demonstrate the capabilities of our tools and discuss how they can be customized to meet your specific needs.

Thank you for considering AI MedReview as a partner in your continued success. I look forward to the opportunity to work together.

Best regards,
[Your Name]
Lead Sales Representative, AI MedReview
[Your Contact Information]
```

---

### Email Draft for Truro PCN

#### Subject: Driving Comprehensive Care with AI MedReview at Truro PCN

#### Email Body:
```
Dear [Clinical Director's Name],

I hope this message finds you in good health. I am reaching out to commend Truro PCN for its innovative approach to health care, including the co-location with various primary and specialist teams and participation in the NHS Cornwall and Isles of Scilly's integrated care initiatives. Your commitment to providing comprehensive care services is truly inspiring.

AI MedReview provides solutions designed to further enhance patient engagement and feedback collection, supporting data-driven decision-making to improve service delivery. Our Friends and Family Test analysis tool captures detailed patient feedback, offering actionable insights that can streamline care coordination and boost patient satisfaction.

We believe AI MedReview can significantly contribute to Truro PCN's goals. I would love to discuss this further and arrange a pilot project to demonstrate the tangible benefits of our tools. Please let me know a convenient time for us to meet and explore how we can tailor our solutions to meet your specific needs.

Thank you for considering this opportunity. I look forward to your response.

Warm regards,
[Your Name]
Lead Sales Representative, AI MedReview
[Your Contact Information]
```

---

### Email Draft for Penwith PCN

#### Subject: Enhancing Patient Feedback and Integrated Care at Penwith PCN

#### Email Body:
```
Dear [Clinical Director's Name],

I hope this email finds you well. I am writing to acknowledge the commendable efforts Penwith PCN has made in adopting integrated care models and engaging in local health initiatives to address community needs. Your dedication to improving primary care services through collaboration is truly noteworthy.

At AI MedReview, we are committed to aiding PCNs like Penwith in enhancing patient feedback analysis and supporting integrated care planning. Our Friends and Family Test analysis tool can streamline the collection of patient feedback, providing actionable insights that can further support your integrated care initiatives.

We would love the opportunity to discuss how our solutions can be customized to meet the specific needs of Penwith PCN. I would be happy to schedule a meeting or demo session at your convenience to demonstrate the capabilities of AI MedReview and explore potential applications within your network.

Thank you for considering AI MedReview as a partner in your continued efforts to improve patient care. I look forward to the opportunity to work together.

Best regards,
[Your Name]
Lead Sales Representative, AI MedReview
[Your Contact Information]
```

---

### Email Draft for East Cornwall PCN

#### Subject: Elevating Service Delivery with AI MedReview at East Cornwall PCN

#### Email Body:
```
Dear [Clinical Director's Name],

I hope this email finds you well. I am writing to commend East Cornwall PCN for its active participation in regional healthcare initiatives and the introduction of new patient care services and support programs. Your focus on delivering high-quality primary care services is truly impressive.

At AI MedReview, we are dedicated to empowering PCNs like East Cornwall with advanced tools for patient feedback collection and analysis. Our Friends and Family Test analysis tool can capture detailed patient insights, providing the data-driven support needed for effective service planning and improvement.

We believe AI MedReview can play a significant role in helping East Cornwall PCN achieve its goals. I would be delighted to schedule a meeting or demo session to showcase how our solutions can be tailored to meet your specific needs and enhance patient care coordination.

Thank you for considering AI MedReview as a partner in your ongoing efforts to improve service delivery. I look forward to the opportunity to collaborate with you.

Best regards,
[Your Name]
Lead Sales Representative, AI MedReview
[Your Contact Information]
```

---

### Email Draft for St Austell Health PCN

#### Subject: Advancing Integrated Care with AI MedReview at St Austell Health PCN

#### Email Body:
```
Dear [Clinical Director's Name],

I hope this email finds you well. I am reaching out to commend St Austell Health PCN for its recent achievements in developing new health initiatives and patient support programs, as well as your collaboration with local healthcare providers to enhance service delivery. Your dedication to providing comprehensive primary care services is truly exemplary.

At AI MedReview, we are committed to supporting PCNs like St Austell Health in collecting and analyzing patient feedback to drive service improvement. Our Friends and Family Test analysis tool can deliver detailed patient insights, offering the data-driven support necessary for planning and executing integrated care services.

We believe AI MedReview can significantly contribute to St Austell Health PCN's objectives. I would be delighted to schedule a meeting or demo session to demonstrate how our solutions can be customized to meet your specific needs and enhance patient care coordination.

Thank you for considering AI MedReview as a partner in your ongoing efforts to improve patient care. I look forward to the opportunity to work together.

Best regards,
[Your Name]
Lead Sales Representative, AI MedReview
[Your Contact Information]
```

---